https://adventofcode.com/2020/day/23

In [1]:
from toolz import take
from collections import Counter

In [2]:
class Node:
    def __init__(self, val, pos=0, prevnode=None, nextnode=None):
        self.val = val
        self.prev = prevnode
        self.next = nextnode
        if self.prev is not None:
            self.prev.next = self
        if self.next is not None:
            self.next.prev = self
            
    def __repr__(self):
        return '<Node %s>' % self.val

        
class LinkedList:
    def __init__(self, head=None):
        self.head = head
    
    @property
    def tail(self):
        if self.head is None:
            raise ValueError("list is empty")
        current = self.head
        while current.next is not None and current.next is not self.head:
            current = current.next
        return current
    
    def find(self, val):
        if self.head is None:
            raise ValueError("list is empty")
        current = self.head
        while current.val != val:
            current = current.next
            if current is self.head or current is None:
                return None
        return current
    
    def __contains__(self, val):
        return self.find(val) is not None
    
    def insert(self, other, insert_after=None):
        if insert_after is None:
            insert_after = self.head
        else:
            other_tail = other.tail
            insert_before = insert_after.next
            insert_after.next = other.head
            other.head.prev = insert_after
            insert_before.prev = other_tail
            other_tail.next = insert_before
    
    def excise(self, begin, end):
        before, after = begin.prev, end.next
        before.next = after
        after.prev = before
        begin.prev = None
        end.next = None
        return LinkedList(begin)
    
    def make_circular(self):
        tail = self.tail
        tail.next = self.head
        self.head.prev = tail

    @classmethod
    def from_list(cls, list):
        it = iter(list)
        head = node = Node(next(it))
        for x in it:
            nextnode = Node(x, prevnode=node)
            node = nextnode
        return cls(head)
    
    def __repr__(self):
        L = []
        node = self.head
        L.append(node)
        while True:
            node = node.next
            if node is None or node is self.head:
                break
            L.append(node)
        return repr(L)
    
    def __iter__(self):
        node = self.head
        while True:
            yield node
            if node.next is self.head or node.next is None:
                break
            node = node.next

In [3]:
cups = LinkedList.from_list([3,2,4,1,5])

In [4]:
cups.make_circular()

In [5]:
cups

[<Node 3>, <Node 2>, <Node 4>, <Node 1>, <Node 5>]

In [6]:
labels = LinkedList.from_list([1,2,3,4,5])
labels.make_circular()
labels

[<Node 1>, <Node 2>, <Node 3>, <Node 4>, <Node 5>]

In [7]:
def countdown(i, mod, minval=1):
    while True:
        i -= 1
        if i < minval:
            i = minval + mod - 1
        yield i

In [8]:
cd = countdown(3, 10, 1)

In [9]:
next(cd)

2

In [10]:
next(countdown(0, 5))

5

In [11]:
cd = countdown(4, 5)

In [12]:
next(cd)

3

In [13]:
def move(cups, n, skip=3, minval=1, debug=False):
    if not skip:
        cups.head = cups.head.next
        return
    
    begin = cups.head.next
    end = begin
    if skip:
        for _ in range(skip-1):
            end = end.next
    frag = cups.excise(begin, end)
    cd = countdown(cups.head.val, n, minval=minval)
    dest_val = next(cd)
    while dest_val in frag:
        dest_val = next(cd)

    destination = cups.find(dest_val)
    cups.insert(frag, destination)
    
    cups.head = cups.head.next


In [14]:
[i for i in range(0)]

[]

In [15]:
vals = [3,8,9,1,2,5,4,6,7]
cups = LinkedList.from_list(vals)
cups.make_circular()
# labels = LinkedList.from_list(sorted(vals))
# labels.make_circular()
cups

[<Node 3>, <Node 8>, <Node 9>, <Node 1>, <Node 2>, <Node 5>, <Node 4>, <Node 6>, <Node 7>]

In [16]:
for _ in range(100):
    move(cups, 9)
cups

[<Node 1>, <Node 6>, <Node 7>, <Node 3>, <Node 8>, <Node 4>, <Node 5>, <Node 2>, <Node 9>]

In [17]:
vals = [5,8,9,1,7,4,2,6,3]
n = len(vals)
cups = LinkedList.from_list(vals)
cups.make_circular()
# labels = LinkedList.from_list(sorted(vals))
# labels.make_circular()
for _ in range(100):
    move(cups, n)
cups

[<Node 4>, <Node 3>, <Node 8>, <Node 9>, <Node 6>, <Node 7>, <Node 2>, <Node 5>, <Node 1>]

## Part 2

In [27]:
%%time
skip = 3
vals = [3,8,9,1,2,5,4,6,7]  # + list(range(10, 1001))
n = len(vals)
minval = min(vals)
cups = LinkedList.from_list(vals)
cups.make_circular()

L = []
for i in range(1_000):
    current = [x.val for x in cups]
    L.append(current)
    move(cups, n, skip=skip, minval=minval)

CPU times: user 3.57 ms, sys: 3.95 ms, total: 7.52 ms
Wall time: 7.44 ms


In [28]:
L[:3]

[[3, 8, 9, 1, 2, 5, 4, 6, 7],
 [2, 8, 9, 1, 5, 4, 6, 7, 3],
 [5, 4, 6, 7, 8, 9, 1, 3, 2]]

In [29]:
M = [x.index(1) for x in L]

In [30]:
N = [i for (i,x) in enumerate(L) if x[0] == 1]

In [32]:
N[:20]

[5,
 11,
 17,
 24,
 38,
 44,
 51,
 62,
 78,
 83,
 97,
 100,
 119,
 129,
 135,
 169,
 180,
 183,
 195,
 200]